# GeoSpatial Data Project - Blasi

TODO's

You recently created a new company in the GAMING industry. The company will have the following scheme:

    20 Designers
    5 UI/UX Engineers
    10 Frontend Developers
    15 Data Engineers
    5 Backend Developers
    20 Account Managers
    1 Maintenance guy that loves basketball
    10 Executives
    1 CEO/President.

As a data engineer you have asked all the employees to show their preferences on where to place the new office. Your goal is to place the new company offices in the best place for the company to grow. You have to find a place that more or less covers all the following requirements (note that it's impossible to cover all requirements, so you have to prioritize at your glance):

    Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.
    30% of the company staff have at least 1 child.
    Developers like to be near successful tech startups that have raised at least 1 Million dollars.
    Executives like Starbucks A LOT. Ensure there's a starbucks not too far.
    Account managers need to travel a lot.
    Everyone in the company is between 25 and 40, give them some place to go party.
    The CEO is vegan.
    If you want to make the maintenance guy happy, a basketball stadium must be around 10 Km.
    The office dog—"Dobby" needs a hairdresser every month. Ensure there's one not too far away.


In [85]:
import requests
import json
import pandas as pd
from getpass import getpass

import os
from dotenv import load_dotenv

from pymongo import MongoClient

import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd




In [86]:
load_dotenv() 

True

In [87]:
employees = ['Designers',' UI/UX Engineers','Frontend Developers','Data Engineers','Backend Developers','Account Managers','Maintenance','Executives',	'CEO/President']
q_employees = [20,	5,	10,	15,	5,	20,	1,	10,	1]

employee_dict = {
    "emp_type":employees,
    "q_emp":q_employees

}
emp = pd.DataFrame(employee_dict)

emp["%_total"] = (emp['q_emp'] / emp['q_emp'].sum())
emp.sort_values("q_emp",ascending=False)

,emp_type,q_emp,%_total
0,Designers,20,0.229885
5,Account Managers,20,0.229885
3,Data Engineers,15,0.172414
2,Frontend Developers,10,0.114943
7,Executives,10,0.114943
1,UI/UX Engineers,5,0.057471
4,Backend Developers,5,0.057471
6,Maintenance,1,0.011494
8,CEO/President,1,0.011494


    Designers: There must be some nearby companies that also do design --> (Company db) design tags
    Developers : Near startups with more than 1M raised -->(Company db) total_money_raised (non numeric format)
    30% of the company staff have at least 1 child. Schools/daycares  (API)
    Executives: Starbucks --> (API)name starbucks
    Account managers: Airport --> (API) name airport
    ALL: party place -->(API) tag party, club?
    CEO: Vegan restaurant --> (API) tag
    maintenance: basketball stadium must be around 10 Km.
    The office dog—"Dobby" needs a hairdresser every month. Ensure there's one not too far away.

## Companies DB - Mongo

In [88]:
client = MongoClient("localhost:27017")
client.list_database_names()
db = client["ironhack"]
c = db.get_collection("companies")

In [89]:
filter = {} 
projection = {"category_code":1}
tag_dict = list(c.find(filter,projection))

tag_list = set([tag_dict[i]["category_code"] for i in range(len(tag_dict))])
tag_list



{None,
 'advertising',
 'analytics',
 'automotive',
 'biotech',
 'cleantech',
 'consulting',
 'design',
 'ecommerce',
 'education',
 'enterprise',
 'fashion',
 'finance',
 'games_video',
 'government',
 'hardware',
 'health',
 'hospitality',
 'legal',
 'local',
 'manufacturing',
 'medical',
 'messaging',
 'mobile',
 'music',
 'nanotech',
 'network_hosting',
 'news',
 'nonprofit',
 'other',
 'photo_video',
 'public_relations',
 'real_estate',
 'search',
 'security',
 'semiconductor',
 'social',
 'software',
 'sports',
 'transportation',
 'travel',
 'web'}

filter = {} 
projection = {"tag_list":1}
tag_dict = list(c.find(filter,projection))

tag_list = set([tag_dict[i]["tag_list"] for i in range(len(tag_dict))])
tag_list

filter = {"tag_list": {"$regex": "(design|games_video|software|web)"}} 
projection ={"name": 1, "offices":1,"tag_list":1,"total_money_raised":1}

pipeline = [
    {"$match": filter},
    {"$project": projection},
    {"$unwind": "$offices"}
]

complete_dict = list(c.aggregate(pipeline))

In [90]:
filter = {"tag_list": {"$regex": "(design|games_video|software|web|internet|digital|gaming|games|animation|UX|UI)"}} 
projection ={"name": 1, "offices":1,"tag_list":1,"total_money_raised":1}

pipeline = [
    {"$match": filter},
    {"$project": projection},
    {"$unwind": "$offices"}
]

complete_dict = list(c.aggregate(pipeline))

doing business



In [91]:
comp_id = [str(complete_dict[i]["_id"]) for i in range(len(complete_dict))]
comp_name = [complete_dict[i]["name"] for i in range(len(complete_dict))]
comp_city = [complete_dict[i]["offices"]["city"] for i in range(len(complete_dict))]
comp_country = [complete_dict[i]["offices"]["country_code"] for i in range(len(complete_dict))]
comp_money_raised = [complete_dict[i]["total_money_raised"] for i in range(len(complete_dict))]
comp_tags = [complete_dict[i]["tag_list"]for i in range(len(complete_dict))]
comp_office_lat = [complete_dict[i]["offices"]["latitude"] for i in range(len(complete_dict))]
comp_office_lon = [complete_dict[i]["offices"]["longitude"] for i in range(len(complete_dict))]



comp_dict= pd.DataFrame({
    "id": comp_id,
    "name" : comp_name,
    "city" : comp_city,
    "country" : comp_country,
    "funds_raised" : comp_money_raised,
    "tags": comp_tags,
    "office_lat": comp_office_lat,
    "office_lon":comp_office_lon
})

comp_df =pd.DataFrame(comp_dict)




### funds raised adjustments

In [92]:
import re 

def convert_value(value):

    amount = float(re.sub(r'[^\d.]', '', value))
    unit = value[-1]

    if unit == 'M':
        amount *= 1000000
    elif unit == 'B':
        amount *= 1000000000
    elif unit == 'k':
        amount *= 1000
    else:
        amount
        
    return amount



In [93]:
comp_df["funds_raised_float"] = comp_df["funds_raised"].apply(lambda x: convert_value(x))
comp_df["currency"] = comp_df["funds_raised"].apply(lambda x: x[0])
comp_df

,id,name,city,country,funds_raised,tags,office_lat,office_lon,funds_raised_float,currency
0,52cdef7c4bab8bd675297d8d,Digg,San Francisco,USA,$45M,"community, social, news, bookmark, digg, techn...",37.764726,-122.394523,45000000.0,$
1,52cdef7c4bab8bd675297d97,Scribd,San Francisco,USA,$25.8M,"book-subscription, digital-library, netflix-fo...",37.789634,-122.404052,25800000.0,$
2,52cdef7c4bab8bd675297da0,Babelgum,London,GBR,$13.2M,"iptv, web2ireland",53.344104,-6.267494,13200000.0,$
3,52cdef7c4bab8bd675297da3,Yahoo!,Sunnyvale,USA,$6.8M,"search, portal, webmail, photos",37.418531,-122.025485,6800000.0,$
4,52cdef7c4bab8bd675297da7,AddThis,Vienna,USA,$73M,"sharing, social-applications, social-media, an...",38.926172,-77.245195,73000000.0,$
...,...,...,...,...,...,...,...,...,...,...
3902,52cdef7f4bab8bd67529c6d4,Treehouse,Portland,USA,$19.4M,"online-education, education, programming, web-...",NaN,NaN,19400000.0,$
3903,52cdef7f4bab8bd67529c6dc,acrossair,London,GBR,$0,"iphone, games, augmented-reality",51.517742,-0.116537,0.0,$
3904,52cdef7f4bab8bd67529c6e4,Getyoo,Brussels,BEL,€1.13M,"social-networks, networking, web, real-life, w...",50.839970,4.346472,1130000.0,€
3905,52cdef7f4bab8bd67529c6f5,Oriact,KÃ¼ssnacht,CHE,$0,"collaboration, communication, appliance, softw...",47.088219,8.437163,0.0,$


In [94]:



ccy = pd.DataFrame({
"sym" : ['$', '€', '£', '¥'],
"fx_rate" : [1,1.07,1.21,0.0076]
})


comp_df = comp_df.merge(ccy,how="left", left_on="currency", right_on="sym")
comp_df["funds_raised_usd"] = comp_df["funds_raised_float"]*comp_df["fx_rate"] # USD equivalent of funds raised


comp_df["desing_tag"] = comp_df["tags"].apply(lambda x: "design" in x) # design tag boolean





In [95]:
comp_df.drop(["funds_raised","sym"],axis=1,inplace=True) # dropping unnecesary columns
comp_df = comp_df[~comp_df["office_lat"].isna()] # filtering for offices with coordinates

## Analysis

- san francisco, London and NY look promising taking into account design firms and startups with more than USD 1M raised

In [96]:
pd.DataFrame(comp_df[["country","city"]][comp_df["funds_raised_usd"]>1000000].value_counts()).head(5) #Q companies with more than USD 1M Raised funds

0
country city             
USA     San Francisco  67
        New York       36
GBR     London         26
USA     Austin         15
        Sunnyvale      13

In [97]:
pd.DataFrame(comp_df[["country","city"]][comp_df["desing_tag"]==True].value_counts()).head(5) #Q companies with design tag

0
country city             
USA     New York       18
GBR     London         14
USA     San Francisco  12
        Los Angeles     9
        Chicago         8

In [98]:
comp_df.head()

,id,name,city,country,tags,office_lat,office_lon,funds_raised_float,currency,fx_rate,funds_raised_usd,desing_tag
0,52cdef7c4bab8bd675297d8d,Digg,San Francisco,USA,"community, social, news, bookmark, digg, techn...",37.764726,-122.394523,45000000.0,$,1.0,45000000.0,True
1,52cdef7c4bab8bd675297d97,Scribd,San Francisco,USA,"book-subscription, digital-library, netflix-fo...",37.789634,-122.404052,25800000.0,$,1.0,25800000.0,False
2,52cdef7c4bab8bd675297da0,Babelgum,London,GBR,"iptv, web2ireland",53.344104,-6.267494,13200000.0,$,1.0,13200000.0,False
3,52cdef7c4bab8bd675297da3,Yahoo!,Sunnyvale,USA,"search, portal, webmail, photos",37.418531,-122.025485,6800000.0,$,1.0,6800000.0,False
4,52cdef7c4bab8bd675297da7,AddThis,Vienna,USA,"sharing, social-applications, social-media, an...",38.926172,-77.245195,73000000.0,$,1.0,73000000.0,False


In [125]:
def heatmap_top3(location,dataframe,condition_filter):
    map_ = Map(location=location, zoom_start = 12)

    filtered_df =comp_df[condition_filter]
    group = folium.FeatureGroup(name=f"Funding {filtered_df.shape[0]}")

    HeatMap(data = filtered_df[["office_lat", "office_lon"]]).add_to(map_)
    group.add_to(map_)
    return map_

In [126]:
heatmap_top3([51.5072178, -0.1275862],comp_df,(comp_df["funds_raised_usd"]>1000000)|(comp_df["desing_tag"]==True)) #LONDON

In [127]:
heatmap_top3([40.7549309, -73.9840195],comp_df,(comp_df["funds_raised_usd"]>1000000)|(comp_df["desing_tag"]==True)) #NY

In [128]:
heatmap_top3([37.7700285, -122.4578586],comp_df,(comp_df["funds_raised_usd"]>1000000)|(comp_df["desing_tag"]==True)) #San Francisco

## Foursquare API

In [133]:
# your code here
ny_lat= 40.7549309
ny_lon=-73.9840195



In [134]:
passwd = os.getenv("passwd")

In [135]:
url = "https://api.foursquare.com/v3/places/search?query=coffee&ll=41.3977421%2C2.1902493&limit=10"

headers = {
    "accept": "application/json",
    "Authorization": passwd
}


response = requests.get(url, headers=headers)


r = response.json()

In [136]:
r

{'results': [{'fsq_id': '530c5e6911d2be0b8d22e27a',
   'categories': [{'id': 13034,
     'name': 'Café',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
      'suffix': '.png'}},
    {'id': 13035,
     'name': 'Coffee Shop',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
      'suffix': '.png'}},
    {'id': 13065,
     'name': 'Restaurant',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
      'suffix': '.png'}}],
   'chains': [],
   'distance': 135,
   'geocodes': {'main': {'latitude': 41.396876, 'longitude': 2.191489},
    'roof': {'latitude': 41.396876, 'longitude': 2.191489}},
   'link': '/v3/places/530c5e6911d2be0b8d22e27a',
   'location': {'address': 'Calle Pamplona, 88',
    'admin_region': 'Cataluña',
    'country': 'ES',
    'cross_street': 'C. de Pallars',
    'formatted_address': 'Calle Pamplona, 88 (C. de Pallars), 08018 Barcelona Catalonia',
    'locality': 'Barcelona',
    'po